# Image and Video Processing: Transfer Learning using TensorFlow

In this notebook, we'll cover the basics of transfer learning for image classification using TensorFlow. We'll use a pre-trained model from `tf.keras.applications` and fine-tune it on a sample dataset.

## What is Transfer Learning?

Transfer learning is a machine learning technique where a model trained on one task is reused on a different but related task. It leverages the knowledge gained from the first task to improve the learning process and performance on the second task.

## Installing Required Libraries

First, we need to install the required libraries. For this tutorial, we'll use `tensorflow` and `tensorflow_datasets`. You can install these libraries using the following commands:

In [1]:
!pip install tensorflow tensorflow-datasets

## Loading the Dataset

We'll use the `tf_flowers` dataset available through the `tensorflow_datasets` library. This dataset consists of 3,670 images of flowers in five categories.

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the tf_flowers dataset
(train_data, test_data), info = tfds.load('tf_flowers', split=['train[:80%]', 'train[80%:]'], with_info=True, as_supervised=True)

print(info)

## Preprocessing the Data

We need to preprocess the images by resizing them to a fixed size and normalizing the pixel values.

In [3]:
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0  # Normalize the image to [0, 1]
    return image, label

# Apply preprocessing to the dataset
train_data = train_data.map(preprocess).cache().shuffle(1000).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

## Building the Model using Transfer Learning

We'll use the pre-trained MobileNetV2 model from `tf.keras.applications` as the base model. We'll add a few layers on top of it and fine-tune the entire model on the `tf_flowers` dataset.

In [4]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')  # 5 classes in the tf_flowers dataset
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Training the Model

We'll train the model using the preprocessed training data. Since the base model is frozen, only the top layers will be trained.

In [5]:
history = model.fit(train_data, epochs=10, validation_data=test_data)

## Fine-tuning the Model

After the initial training, we can unfreeze the base model and fine-tune it along with the top layers for a few more epochs.

In [6]:
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_fine = model.fit(train_data, epochs=5, validation_data=test_data)

## Evaluating the Model

After training and fine-tuning the model, we need to evaluate its performance on the test dataset.

In [7]:
loss, accuracy = model.evaluate(test_data)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

## Conclusion

In this notebook, we covered the basics of transfer learning for image classification using TensorFlow. We used the pre-trained MobileNetV2 model from `tf.keras.applications`, fine-tuned it on the `tf_flowers` dataset, and evaluated its performance. Transfer learning is a powerful technique that allows us to leverage pre-trained models and achieve high performance with limited data and computational resources.